In [0]:
# from pyspark.ml import PipelineModel
# from pyspark.sql.functions import col, udf
# from pyspark.ml.linalg import DenseVector, SparseVector
# from pyspark.sql.types import ArrayType, DoubleType
# import pandas as pd
# import numpy as np
# import os

# from catboost import CatBoostClassifier
# from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
# from sklearn.model_selection import GridSearchCV
# import matplotlib.pyplot as plt
# import seaborn as sns

# # ================================
# # 1. Load top-k features slicer model
# # ================================
# slicer_model = PipelineModel.load("/FileStore/models/slicer_top10")

# train_ready = spark.read.format("delta").load("/FileStore/data/train_ready")
# val_ready = spark.read.format("delta").load("/FileStore/data/val_ready")

# # ================================
# # 2. Apply slicer to datasets
# # ================================
# train_topk = slicer_model.transform(train_ready)
# val_topk = slicer_model.transform(val_ready)

# # ================================
# # 3. Light undersampling
# # ================================
# minority_df = train_topk.filter(col("label") == 1)
# majority_df = train_topk.filter(col("label") != 1)
# train_balanced = majority_df.sample(False, 0.1, seed=42).union(minority_df)

# # ================================
# # 4. Convert VectorUDT to list
# # ================================
# def vector_to_array(v):
#     if isinstance(v, DenseVector) or isinstance(v, SparseVector):
#         return v.toArray().tolist()
#     return v

# vector_to_array_udf = udf(vector_to_array, ArrayType(DoubleType()))

# train_array = train_balanced.withColumn("features_array", vector_to_array_udf("features_topK"))
# val_array = val_topk.withColumn("features_array", vector_to_array_udf("features_topK"))

# # ================================
# # 5. Convert to Pandas
# # ================================
# train_pd = train_array.select("features_array", "label").toPandas()
# val_pd = val_array.select("features_array", "label").toPandas()

# X_train = pd.DataFrame(train_pd["features_array"].tolist())
# y_train = train_pd["label"]

# X_val = pd.DataFrame(val_pd["features_array"].tolist())
# y_val = val_pd["label"]

# # ================================
# # 6. Class Weights
# # ================================
# label_counts = y_train.value_counts().to_dict()
# total = sum(label_counts.values())
# class_weights = {label: total / count for label, count in label_counts.items()}
# class_weights_list = [class_weights[i] for i in sorted(class_weights)]

# # ================================
# # 7. Hyperparameter Tuning (GridSearch)
# # ================================
# params = {
#     'depth': [4, 6, 8],
#     'learning_rate': [0.03, 0.05, 0.1],
#     'iterations': [100, 200],
#     'l2_leaf_reg': [1, 3, 5],
# }

# cat = CatBoostClassifier(
#     loss_function='MultiClass',
#     class_weights=class_weights_list,
#     verbose=0,
#     random_seed=42
# )

# grid = GridSearchCV(cat, param_grid=params, cv=3, scoring='f1_macro', n_jobs=-1)
# grid.fit(X_train, y_train)

# # Best model
# model = grid.best_estimator_
# print("✅ Best Parameters Found:", grid.best_params_)

# # ================================
# # 8. Fit with Early Stopping
# # ================================
# model.set_params(early_stopping_rounds=50, verbose=50)
# model.fit(X_train, y_train, eval_set=(X_val, y_val))

# # ================================
# # 9. Evaluation
# # ================================
# y_pred = model.predict(X_val)
# y_proba = model.predict_proba(X_val)

# print("\n📊 Classification Report:")
# print(classification_report(y_val, y_pred, digits=4))

# print("📉 Confusion Matrix:")
# print(confusion_matrix(y_val, y_pred))

# # AUC por classe
# print("\n🎯 AUC Score por classe:")
# for i in range(y_proba.shape[1]):
#     auc = roc_auc_score((y_val == i).astype(int), y_proba[:, i])
#     print(f"Classe {i}: AUC = {auc:.4f}")

# # ================================
# # 10. Feature Importance Plot
# # ================================
# feat_imp = model.get_feature_importance()
# plt.figure(figsize=(8, 5))
# sns.barplot(x=feat_imp, y=[f'feature_{i}' for i in range(len(feat_imp))])
# plt.title("Feature Importance (Top-10)")
# plt.xlabel("Importance")
# plt.ylabel("Feature")
# plt.tight_layout()
# plt.show()

# # ================================
# # 11. Save Model
# # ================================
# os.makedirs("/dbfs/FileStore/models/", exist_ok=True)
# model.save_model("/dbfs/FileStore/models/cb_top10_model_tuned.cbm")

✅ Best Parameters Found: {'depth': 8, 'iterations': 200, 'l2_leaf_reg': 1, 'learning_rate': 0.1}


---------------------------------------------------------------------------
CatBoostError                             Traceback (most recent call last)
File <command-2292840792669070>:96
     91 print("✅ Best Parameters Found:", grid.best_params_)
     93 # ================================
     94 # 8. Fit with Early Stopping
     95 # ================================
---> 96 model.set_params(early_stopping_rounds=50, verbose=50)
     97 model.fit(X_train, y_train, eval_set=(X_val, y_val))
     99 # ================================
    100 # 9. Evaluation
    101 # ================================

File /local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.9/site-packages/catboost/core.py:3537, in CatBoost.set_params(self, **params)
   3528 """
   3529 Set parameters into CatBoost model.
   3530 
   (...)
   3534     List of key=value paris. Example: model.set_params(iterations=500, thread_count=2).
   3535 """
   3536 if self.is_fitted():
-> 3537     raise CatBoostError("Yo

In [0]:
from pyspark.ml import PipelineModel
from pyspark.sql.functions import col, udf
from pyspark.ml.linalg import DenseVector, SparseVector
from pyspark.sql.types import ArrayType, DoubleType
import pandas as pd
import numpy as np
import os

from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score, balanced_accuracy_score, recall_score
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

# ================================
# 1. Load top-k features slicer model
# ================================
slicer_model = PipelineModel.load("/FileStore/models/slicer_top10")

train_ready = spark.read.format("delta").load("/FileStore/data/train_ready")
val_ready = spark.read.format("delta").load("/FileStore/data/val_ready")

# ================================
# 2. Apply slicer to datasets
# ================================
train_topk = slicer_model.transform(train_ready)
val_topk = slicer_model.transform(val_ready)

# ================================
# 3. Light undersampling
# ================================
minority_df = train_topk.filter(col("label") == 1)
majority_df = train_topk.filter(col("label") != 1)
train_balanced = majority_df.sample(False, 0.8, seed=42).union(minority_df)

# ================================
# 4. Convert VectorUDT to list
# ================================
def vector_to_array(v):
    if isinstance(v, DenseVector) or isinstance(v, SparseVector):
        return v.toArray().tolist()
    return v

vector_to_array_udf = udf(vector_to_array, ArrayType(DoubleType()))

train_array = train_balanced.withColumn("features_array", vector_to_array_udf("features_topK"))
val_array = val_topk.withColumn("features_array", vector_to_array_udf("features_topK"))

# ================================
# 5. Convert to Pandas
# ================================
train_pd = train_array.select("features_array", "label").toPandas()
val_pd = val_array.select("features_array", "label").toPandas()

X_train = pd.DataFrame(train_pd["features_array"].tolist())
y_train = train_pd["label"]

X_val = pd.DataFrame(val_pd["features_array"].tolist())
y_val = val_pd["label"]

# ================================
# 6–7. Manual Grid Search for Class Weights
# ================================
best_score = -1
best_weights = None
best_model = None

weight_options = [11, 12, 13, 14, 15, 16, 17, 18, 19]
results = []

print("🔎 Iniciando grid search para class weights...")

for w1 in weight_options:
    class_weights_list = [1, w1]  # Classe 0 → 1, Classe 1 → w1
    
    model = CatBoostClassifier(
        depth=8,
        iterations=200,
        learning_rate=0.1,
        l2_leaf_reg=1,
        loss_function='MultiClass',
        class_weights=class_weights_list,
        early_stopping_rounds=50,
        verbose=0,
        random_seed=42
    )

    model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=0)
    
    y_pred = model.predict(X_val)
    y_pred = np.array(y_pred).ravel()  # Garantir array 1D
    
    unique_preds = np.unique(y_pred)
    if len(unique_preds) < 2:
        print(f"⚠️ Ignorado class_weights={class_weights_list} → só previu a classe {unique_preds[0]}")
        continue
    
    score_f1 = f1_score(y_val, y_pred, average='macro')
    score_bal = balanced_accuracy_score(y_val, y_pred)
    score_rec = recall_score(y_val, y_pred, average='macro')
    
    results.append((class_weights_list, score_f1, score_bal, score_rec, model))
    
    print(f"✔️ class_weights={class_weights_list} | F1-macro: {score_f1:.4f} | Balanced Acc: {score_bal:.4f} | Recall-macro: {score_rec:.4f} | Pred distrib: {Counter(y_pred)}")

    
    if score_f1 > best_score:
        best_score = score_f1
        best_weights = class_weights_list
        best_model = model

print("\n🔍 Melhor resultado:")
print(f"Class weights: {best_weights}")
print(f"F1-macro: {best_score:.4f}")

# ================================
# 8. Avaliação do melhor modelo
# ================================
y_pred = best_model.predict(X_val)
y_proba = best_model.predict_proba(X_val)

print("\n📊 Classification Report:")
print(classification_report(y_val, y_pred, digits=4))

print("📉 Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))

print("\n🎯 AUC Score por classe:")
for i in range(y_proba.shape[1]):
    auc = roc_auc_score((y_val == i).astype(int), y_proba[:, i])
    print(f"Classe {i}: AUC = {auc:.4f}")

# ================================
# 9. Save Best Model
# ================================
os.makedirs("/dbfs/FileStore/models/", exist_ok=True)
best_model.save_model("/dbfs/FileStore/models/cb_top10_model_best_weights.cbm")

🔎 Iniciando grid search para class weights...
✔️ class_weights=[1, 11] | F1-macro: 0.4909 | Balanced Acc: 0.5000 | Recall-macro: 0.5000 | Pred distrib: Counter({0.0: 740542, 1.0: 77})
✔️ class_weights=[1, 12] | F1-macro: 0.4913 | Balanced Acc: 0.5001 | Recall-macro: 0.5001 | Pred distrib: Counter({0.0: 740373, 1.0: 246})
✔️ class_weights=[1, 13] | F1-macro: 0.4917 | Balanced Acc: 0.5002 | Recall-macro: 0.5002 | Pred distrib: Counter({0.0: 740112, 1.0: 507})
✔️ class_weights=[1, 14] | F1-macro: 0.4942 | Balanced Acc: 0.5007 | Recall-macro: 0.5007 | Pred distrib: Counter({0.0: 738365, 1.0: 2254})
✔️ class_weights=[1, 15] | F1-macro: 0.4957 | Balanced Acc: 0.5007 | Recall-macro: 0.5007 | Pred distrib: Counter({0.0: 736230, 1.0: 4389})
✔️ class_weights=[1, 16] | F1-macro: 0.5015 | Balanced Acc: 0.5021 | Recall-macro: 0.5021 | Pred distrib: Counter({0.0: 725974, 1.0: 14645})
✔️ class_weights=[1, 17] | F1-macro: 0.4864 | Balanced Acc: 0.5159 | Recall-macro: 0.5159 | Pred distrib: Counter({0.